In the old XSPEC fit files, the parameter names are not contained within the FITS files, so we have to read from the JSON files to get these. The FITS files simply have 'PARAM1', 'PARAM2', etc. The new XSPEC fitting files will have the parameter names in the header of the table that holds the parameters. 

In the old files, the parameter errors are confidence intervals and they are margins of error in the new files. 

In [1]:
from __future__ import division
from collections import OrderedDict
import os
import json

import numpy as np
from astropy.io import fits as pyfits

from grbTools.XSPEC.uncertainty import Calc_Uncertainty

from grbTools.XSPEC.uncertainty import Calc_Eiso_Uncertainty
from grbTools.XSPEC.uncertainty import Calc_Fluence_Uncertainty 
from grbTools.XSPEC.uncertainty import Calc_Flux_Uncertainty

## A single GRB's fit result. 

In [2]:
burst = 'bn100728095'  # just for a reminder

model = 'grbm'
det = 'L'
version = '-01-'
dur = 165.378
z = 1.567
energetic = 'flux'


# *~*~*~ Find appropriate files. 
detDir = ('GBMwLAT' if 'L' in det else 'GBM')
paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
             '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                      version, det))
covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
             '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                     version, det))
# *~*~*~ Covaraince Matrix
f1 = pyfits.open(covarfile)
COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
nPars = int(np.sqrt(len(COVARMAT)))
COVARMAT = COVARMAT.reshape((nPars, nPars))
#COVARMAT = np.asmatrix(COVARMAT)
f1.close()
del f1

# *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
parameters = json.load(open(paramfile, 'r'), 
                       object_pairs_hook=OrderedDict)[0] 
# Keep only the first value, which is param value
for key in parameters.keys():
    parameters[key] = parameters[key][0]

# *~*~*~ Calculate Uncertainty
unc = Calc_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT, 
                       energetic=energetic)
print(unc)


# *~*~*~ Calculate Uncertainty
unc = Calc_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT, 
                       energetic='fluence')
print(unc)

# *~*~*~ Calculate Uncertainty
unc = Calc_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT, 
                       energetic='eiso')
print(unc)

2.78623310577e-08
4.60781658567e-06
2.9982030925e+52


### Using the Flux Specific Function:

In [3]:
unc = Calc_Flux_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT)
print(unc)

2.78623310577e-08


### Using the Fluence Specific Function:

In [4]:
unc = Calc_Fluence_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT)
print(unc)

4.60781658567e-06


### Using the Eiso Specific Function:

In [5]:
unc = Calc_Eiso_Uncertainty(model=model, 
                       parameters=parameters, 
                       emin=10., 
                       emax=10000., 
                       redshift=z, 
                       duration=dur,
                       covarmat=COVARMAT)
print(unc)

2.9982030925e+52


## Every model fit result for a single GRB.

In [6]:
models = '''cutoffpl
cutoffpl+bbody
cutoffpl+bbody+lpow
cutoffpl+lpow
grbm
grbm+bbody
grbm+bbody+lpow
grbm+lpow
sbpl
sbpl+bbody
sbpl+bbody+lpow
sbpl+lpow'''

models = models.split('\n')

In [7]:
burst = 'bn100728095'  # just for a reminder

det = 'L'
version = '-01-'
dur = 165.378
z = 1.567
energetic = 'flux'

In [8]:
for model in models:
    print(model)
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    fluxfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                '%s/%s/%s_%s_%s_bestfit_Flux_PYXSPECFIT.json'%(burst, detDir, model, model,
                                                               version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))

    if model == 'grbm+bbody':
        # -01- versions have too low of kT. Math domain error. 
        paramfile = paramfile.replace('-01-','-02-')
        fluxfile = fluxfile.replace('-01-','-02-')
        covarfile = covarfile.replace('-01-','-02-')
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    #COVARMAT = np.asmatrix(COVARMAT)
    f1.close()
    del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    # Keep only the first value, which is param value
    for key in parameters.keys():
        parameters[key] = parameters[key][0]
    
    
    # *~*~*~ fluxes, fluence, eiso from file. Compare this calc agains old. 
    #  WARNING: eisos won't match files since we updated cosmology since then. 
    energetics = json.load(open(fluxfile, 'r'), 
                           object_pairs_hook=OrderedDict)
    if energetic == 'flux':
        eng = energetics['Energy_Flux_K']
    elif energetic == 'fluence':
        eng = energetics['Fluence']
    elif energetic == 'eiso':
        eng = energetics['Eiso']
    print(eng[0]-eng[1])  # old unc
    print(eng[2]-eng[0])  # old unc
    
    # *~*~*~ Calculate Uncertainty
    unc = Calc_Uncertainty(model=model, 
                           parameters=parameters, 
                           emin=10., 
                           emax=10000., 
                           redshift=z, 
                           duration=dur,
                           covarmat=COVARMAT, 
                           energetic=energetic)
    print(unc)  # print new unc
    print('')

cutoffpl
1.32093907272e-08
1.32093907272e-08
1.32093907272e-08

cutoffpl+bbody
2.43832406702e-08
2.43832406702e-08
2.43832406702e-08

cutoffpl+bbody+lpow
2.43815558334e-08
2.43815558334e-08
2.43815558334e-08

cutoffpl+lpow
1.32079983975e-08
1.32079983975e-08
1.32079983975e-08

grbm
2.78623310575e-08
2.78623310575e-08
2.78623310577e-08

grbm+bbody
4.34433985589e-08
4.34433985589e-08
4.34433985744e-08

grbm+bbody+lpow
4.03312493129e-08
4.03312493129e-08
4.03312493103e-08

grbm+lpow
3.52274580545e-08
3.52274580545e-08
3.52274581254e-08

sbpl
2.40782499681e-08
2.40782499681e-08
2.40782499681e-08

sbpl+bbody
3.77873192884e-08
3.77873192884e-08
3.77873192884e-08

sbpl+bbody+lpow
4.1339318717e-08
4.1339318717e-08
4.1339318717e-08

sbpl+lpow
2.60522932102e-08
2.60522932102e-08
2.60522932102e-08



# Same model, different GRBs

In [9]:
bursts = '''bn080916009
bn090323002
bn090328401
bn090510016
bn090902462
bn090926181
bn091003191
bn091208410
bn100728095
bn110731465
bn130518580
bn131108862
bn131231198'''
bursts = bursts.split('\n')

#### Load burst data so that you have respective redshifts and durations. 

In [10]:
import pandas as pd

burst_data = pd.read_csv('/Users/KimiZ/GRBs2/Sample/bursts.txt', sep=' ', header=0)
burst_data['duration'] = burst_data.t90_stop-burst_data.t90_start
burst_data.head()

,number,name,trigger,z,t90_start,t90_stop,duration
0,1,GRB080804,bn080804972,2.2045,0.256,24.960,24.704
1,2,GRB080810,bn080810549,3.3604,-20.096,87.361,107.457
2,3,GRB080905A,bn080905499,0.1218,-0.064,0.896,0.960
3,4,GRB080905B,bn080905705,2.3739,-5.120,100.864,105.984
4,5,GRB080916A,bn080916406,0.6887,0.512,46.849,46.337


In [11]:
model = 'grbm'
det = 'L'
version = '-01-'
energetic = 'flux'

In [12]:
for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)
    print(burst, z, dur)
    
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    fluxfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                '%s/%s/%s_%s_%s_bestfit_Flux_PYXSPECFIT.json'%(burst, detDir, model, model,
                                                               version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    #COVARMAT = np.asmatrix(COVARMAT)
    f1.close()
    del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    # Keep only the first value, which is param value
    for key in parameters.keys():
        parameters[key] = parameters[key][0]
    
    
    # *~*~*~ fluxes, fluence, eiso from file. Compare this calc agains old. 
    #  WARNING: eisos won't match files since we updated cosmology since then. 
    energetics = json.load(open(fluxfile, 'r'), 
                           object_pairs_hook=OrderedDict)
    if energetic == 'flux':
        eng = energetics['Energy_Flux_K']
    elif energetic == 'fluence':
        eng = energetics['Fluence']
    elif energetic == 'eiso':
        eng = energetics['Eiso']
    print(eng[0]-eng[1])  # old unc
    print(eng[2]-eng[0])  # old unc
    
    # *~*~*~ Calculate Uncertainty
    unc = Calc_Uncertainty(model=model, 
                           parameters=parameters, 
                           emin=10., 
                           emax=10000., 
                           redshift=z, 
                           duration=dur,
                           covarmat=COVARMAT, 
                           energetic=energetic)
    print(unc)  # print new unc
    print('')

('bn080916009', 4.35, 62.977000000000004)
3.04822964062e-08
3.04822964062e-08
3.04822964021e-08

('bn090323002', 3.57, 135.17)
2.49101855689e-08
2.49101855689e-08
2.49101855521e-08

('bn090328401', 0.736, 61.69699999999999)
3.68103966995e-08
3.68103966995e-08
3.68103966744e-08

('bn090510016', 0.903, 0.9600000000000001)
8.96090865252e-07
8.96090865252e-07
8.96090863562e-07

('bn090902462', 1.8219999999999998, 19.328000000000003)
1.58433915217e-07
1.58433915217e-07
1.58433915122e-07

('bn090926181', 2.1062, 13.76)
6.94019924592e-08
6.94019924592e-08
6.94019924502e-08

('bn091003191', 0.8969, 20.223999999999997)
5.748218709e-08
5.748218709e-08
5.74821871007e-08

('bn091208410', 1.0633, 12.48)
6.45414356206e-08
6.45414356206e-08
6.45414356222e-08

('bn100728095', 1.567, 165.378)
2.78623310575e-08
2.78623310575e-08
2.78623310577e-08

('bn110731465', 2.83, 7.484999999999999)
7.29703111307e-08
7.29703111307e-08
7.29703111492e-08

('bn130518580', 2.49, 48.577)
3.74941673071e-08
3.74941673071e

# Epeak Uncertainty

In [13]:
from grbTools.XSPEC.uncertainty import Calc_Epeak_Uncertainty

In [14]:
model = 'grbm'




for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    #COVARMAT = np.asmatrix(COVARMAT)
    f1.close()
    del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    # Keep only the first value, which is param value
    for key in parameters.keys():
        parameters[key] = parameters[key][0]
        
    epeak, epeakUnc = Calc_Epeak_Uncertainty(model=model, 
                                             parameters=parameters, 
                                             covarmat=COVARMAT)
    print('%s:  %6.1f +- %6.1f'%(burst, epeak, epeakUnc))
    
    
    


bn080916009:   510.6 +-   34.3
bn090323002:   702.3 +-   53.2
bn090328401:   635.2 +-   51.8
bn090510016:  2052.3 +-  331.1
bn090902462:  1020.8 +-   18.4
bn090926181:   302.7 +-    4.0
bn091003191:   387.7 +-   23.4
bn091208410:   104.3 +-   17.7
bn100728095:   298.1 +-    9.8
bn110731465:   256.5 +-   14.1
bn130518580:   422.1 +-   14.7
bn131108862:   330.4 +-   17.0
bn131231198:   220.4 +-    3.4


In [15]:
model = 'sbpl'



for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    #COVARMAT = np.asmatrix(COVARMAT)
    f1.close()
    del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    # Keep only the first value, which is param value
    for key in parameters.keys():
        parameters[key] = parameters[key][0]
        
    epeak, epeakUnc = Calc_Epeak_Uncertainty(model=model, 
                                             parameters=parameters, 
                                             covarmat=COVARMAT)
    print('%s:  %6.1f +- %6.1f'%(burst, epeak, epeakUnc))
    


bn080916009:   456.2 +-   19.8
bn090323002:   505.0 +-   27.6
bn090328401:   536.8 +-   31.1
bn090510016:  1866.2 +-  228.7
bn090902462:   780.8 +-   11.0
bn090926181:   288.7 +-    3.0
bn091003191:   324.3 +-   15.6
bn091208410:    85.8 +-   12.2
bn100728095:   257.1 +-    6.2
bn110731465:   248.3 +-   11.5
bn130518580:   337.3 +-    8.9
bn131108862:   335.0 +-   12.5
bn131231198:   188.7 +-    2.5


# ASYMMETRIC ERRORS

In [16]:
from grbTools.XSPEC.uncertainty import Calc_Epeak_Uncertainty_Assym

In [17]:
model = 'grbm'



for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    f1.close()
    #del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    values=[]
    for col in f1[2].columns.names:
        if 'PARAM' in col:
            val,lo,up = f1[2].data[col][0][0:3]
            values.append([val, val-lo, up-val])
    for i,key in enumerate(parameters.keys()):
        parameters[key] = values[i]
    COVARMAT_LO = COVARMAT.copy()
    COVARMAT_UP = COVARMAT.copy()

    for i,key in enumerate(parameters.keys()):
        COVARMAT_LO[i][i] = parameters[key][1]**2
        COVARMAT_UP[i][i] = parameters[key][2]**2

    epeak, epeakUnc_LO, epeakUnc_UP = Calc_Epeak_Uncertainty_Assym(model=model, 
                                             parameters=parameters, 
                                             covarmat_lo=COVARMAT_LO,
                                             covarmat_up=COVARMAT_UP)
    print('%s:  %6.1f   - %6.1f + %6.1f'%(burst, epeak, epeakUnc_LO, epeakUnc_UP))
  

bn080916009:   510.6   -   31.9 +   38.0
bn090323002:   702.3   -   46.5 +   54.4
bn090328401:   635.2   -   47.2 +   58.7
bn090510016:  2052.3   -  251.9 +  334.6
bn090902462:  1020.8   -   14.9 +   14.5
bn090926181:   302.7   -    3.6 +    4.0
bn091003191:   387.7   -   20.7 +   25.3
bn091208410:   104.3   -   17.1 +   33.0
bn100728095:   298.1   -    9.5 +   11.1
bn110731465:   256.5   -    8.8 +   11.8
bn130518580:   422.1   -   15.0 +   17.0
bn131108862:   330.4   -   13.8 +   17.4
bn131231198:   220.4   -    3.7 +    3.9


In [18]:
model = 'sbpl'



for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    f1.close()
    #del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    values=[]
    for col in f1[2].columns.names:
        if 'PARAM' in col:
            val,lo,up = f1[2].data[col][0][0:3]
            values.append([val, val-lo, up-val])
    for i,key in enumerate(parameters.keys()):
        parameters[key] = values[i]
    COVARMAT_LO = COVARMAT.copy()
    COVARMAT_UP = COVARMAT.copy()

    for i,key in enumerate(parameters.keys()):
        COVARMAT_LO[i][i] = parameters[key][1]**2
        COVARMAT_UP[i][i] = parameters[key][2]**2

    epeak, epeakUnc_LO, epeakUnc_UP = Calc_Epeak_Uncertainty_Assym(model=model, 
                                             parameters=parameters, 
                                             covarmat_lo=COVARMAT_LO,
                                             covarmat_up=COVARMAT_UP)
    print('%s:  %6.1f   - %6.1f + %6.1f'%(burst, epeak, epeakUnc_LO, epeakUnc_UP))
  

bn080916009:   456.2   -   18.5 +   22.0
bn090323002:   505.0   -   23.2 +   27.4
bn090328401:   536.8   -   30.1 +   36.2
bn090510016:  1866.2   -  218.5 +  278.3
bn090902462:   780.8   -    9.9 +   10.7
bn090926181:   288.7   -    3.1 +    3.1
bn091003191:   324.3   -   17.9 +   19.5
bn091208410:    85.8   -   40.4 +   55.4
bn100728095:   257.1   -    1.5 +    nan
bn110731465:   248.3   -    7.2 +    8.8
bn130518580:   337.3   -   10.2 +   10.9
bn131108862:   335.0   -   11.0 +   12.6
bn131231198:   188.7   -    2.0 +    2.1


/Users/KimiZ/Python/My_Modules/grbTools/XSPEC/uncertainty.py:442: RuntimeWarning: invalid value encountered in sqrt
  uncertainty_up = float(np.sqrt(variance_up))
